In [ ]:
!pip install ultralytics opencv-python -q

In [ ]:
%cd yolov5
%pip install -qr requirements.txt   # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-422-g2540fd4c Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.3/8062.4 GB disk)


# Define you paths here


In [ ]:
weights_path = "/kaggle/input/fusedmodel/yolov12m_fusion_payload.pt" # model path
images_path = "/kaggle/input/vip-cup-test-payload-chunk-1/RGB/images" # images directory for test
device = "0,1" # for multiGPU inference - 0,1 | for single GPU - 0 | for cpu - "cpu"
submission_filename = "submission_fusion_Payload.csv"
output_dir = "/kaggle/working/output" # save results

devices = ["0,1", "cpu"]   

In [3]:
%cd ..

/kaggle/working


In [ ]:
%%time
!python /yolov5/detect.py \
    --weights "$weights_path" \
    --source "$images_path" \
    --img 640 \
    --conf 0.1 \
    --save-txt \
    --project "$output_dir" \
    --device "$device" > /dev/null 2>&1 \
    --save-conf

In [ ]:
import glob
import pandas as pd
import os

rows = []
labels_dir = f"{output_dir}/labels"
for txt_path in glob.glob(os.path.join(labels_dir, "*.txt")):
    # Get image file name (replace .txt with .jpg)
    image_name = os.path.basename(txt_path).replace(".txt", ".jpg")

    with open(txt_path) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 6:
                cls, xc, yc, w, h, conf = map(float, parts)
            else:
                # YOLOv5 format without conf — fallback (e.g., exported from train set)
                cls, xc, yc, w, h = map(float, parts[:5])
                conf = 1.0

            prob_harmful = conf if int(cls) == 0 else 0.0
            payload_label = "Harmful" if int(cls) == 0 else "Normal"

            x_min = xc - w / 2
            y_min = yc - h / 2
            x_max = xc + w / 2
            y_max = yc + h / 2

            rows.append({
                "Frame_name": image_name,
                "track_id": 0,
                "x_min_norm": f"{x_min:.4f}",
                "y_min_norm": f"{y_min:.4f}",
                "x_max_norm": f"{x_max:.4f}",
                "y_max_norm": f"{y_max:.4f}",
                "class_label": 0,
                "direction": 0,
                "confidence_detection": 0,
                "inference_time_detection (ms)": 0,
                "FPS (CPU)": 0,
                "FPS (GPU)": 0,
                "confidence_track": 0,
                "payload_label": payload_label,
                "prob_harmful": f"{prob_harmful:.4f}"
            })

# Save final submission CSV
df = pd.DataFrame(rows)
df.to_csv(f"{submission_filename}", index=False)
print(f"✅ Submission CSV saved as {submission_filename}")


✅ Submission CSV saved as FUSEDsubmissionRGBpayload.csv
